In this notebook we learn the following things

1. Initializing the OPENAI 4o model using langchain
2. Simply calling with a query and getting an answer
3. Chaining multiple queries and feeding the answer of one to the next
4. Using agents to help with out query like wikipedia, calculator e.t.c
5. Storing context in terms of memory
6. Limiting the memory so that it doesn't cost as much

In [2]:
from secret_key import openapi_key
import os

os.environ['OPENAI_API_KEY'] = openapi_key
print(openapi_key)
print(os.environ['OPENAI_API_KEY'])

sk-proj-yWUN27rfHGp4MFj2Du7pT3BlbkFJKoo3wUUGEYWdGpy5raHT
sk-proj-yWUN27rfHGp4MFj2Du7pT3BlbkFJKoo3wUUGEYWdGpy5raHT


In [ ]:
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

In [54]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4o', temperature=0.9)
question = "I want to open a restaurant for indian food. Suggest a fancy name for this."

name = llm.invoke(question)
print(name)

content="That sounds like an exciting venture! Here are some fancy name suggestions for your Indian restaurant:\n\n1. **Maharaja's Feast**\n2. **Saffron Palace**\n3. **Royal Spice Garden**\n4. **Taj Temptations**\n5. **Golden Peacock**\n6. **Curry Couture**\n7. **Spice Symphony**\n8. **Eternal Flavors**\n9. **Indigo Essence**\n10. **Nirvana Nook**\n\nThese names aim to evoke a sense of elegance, luxury, and the rich culinary tradition of Indian cuisine. Good luck with your restaurant!" response_metadata={'token_usage': {'completion_tokens': 122, 'prompt_tokens': 24, 'total_tokens': 146}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None} id='run-d8d37f75-5b0c-4ba3-b1d8-642b5c11a57e-0'


In [16]:
print(name.content)

Sure! Here are a few elegant and sophisticated names you might consider for your Indian restaurant:

1. **Spice Symphony**
2. **Royal Rasoi**
3. **Mughal Majesty**
4. **Nawab's Nest**
5. **Taj Taste**
6. **Saffron Sanctuary**
7. **The Maharaja's Table**
8. **Pashmina Palace**
9. **Diwan Dining**
10. **Bollywood Bistro**

These names evoke a sense of luxury and authenticity, making them perfect for a high-end Indian dining experience.


In [25]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest only one name and say nothing else."
)

prompt_template_name.format(cuisine="Mexican")

'I want to open a restaurant for Mexican food. Suggest only one name and say nothing else.'

In [26]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
answer = chain.run("Italian")

In [27]:
answer

'La Dolce Vita'

In [31]:
prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. return ut as a comma separated list"
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [32]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [chain, food_items_chain])
response = chain.run("Mexican")
print(response)

Tacos al Pastor, Carne Asada Tacos, Chicken Tinga Tacos, Veggie Tacos, Beef Enchiladas, Cheese Enchiladas, Chicken Enchiladas, Shrimp Enchiladas, Carnitas Burrito, Bean and Cheese Burrito, Chicken Fajitas, Steak Fajitas, Shrimp Fajitas, Chiles Rellenos, Tamales, Chicken Quesadilla, Beef Quesadilla, Veggie Quesadilla, Guacamole and Chips, Nachos, Mexican Street Corn, Sopa de Tortilla, Pozole, Menudo, Horchata, Tamarindo Agua Fresca, Churros, Flan, Tres Leches Cake


In [40]:
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest only one name and say nothing else."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key='restaurant_name')

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. return ut as a comma separated list"
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key='menu_items')

In [42]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

chain.invoke({'cuisine': 'Arabic'})

{'cuisine': 'Arabic',
 'restaurant_name': 'Sahara Delights',
 'menu_items': 'Sure! Here are some menu items for Sahara Delights:\n\n1. Moroccan Chicken Tagine,\n2. Lamb Couscous,\n3. Spicy Harira Soup,\n4. Falafel Wraps,\n5. Chicken Shawarma Platter,\n6. Grilled Vegetable Skewers,\n7. Hummus with Pita,\n8. Beef Kofta Kebabs,\n9. Baba Ghanoush,\n10. Tabbouleh Salad,\n11. Mint Tea,\n12. Baklava,\n13. Stuffed Grape Leaves,\n14. Moroccan Mint Lemonade,\n15. Almond Briouats,\n16. Spiced Lentil Soup,\n17. Zaalouk (Eggplant Salad),\n18. Chicken Bastilla,\n19. Spiced Lamb Meatballs,\n20. Rosewater Rice Pudding.'}

In [55]:
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(["llm-math"], llm=llm)

In [56]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools.append(wikipedia)

In [57]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [58]:
agent.run("When was elon musk born and what is his age right now in 2024? be accurate in calculating months")



> Entering new AgentExecutor chain...
To answer this question, I need to find Elon Musk's birth date and then calculate his age as of a specific date in 2024.

Step 1: Find Elon Musk's date of birth.
Step 2: Calculate Elon Musk's age in 2024, including months for accuracy.

Let's start with Step 1.

Action: wikipedia
Action Input: Elon Musk birth date
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. 
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Preto

'Elon Musk was born on June 28, 1971. As of April 23, 2024, he would be 52 years old and 9 months and 26 days.'

In [59]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
answer = chain.run("Italian")
print(answer)

La Bella Cucina


In [60]:
answer = chain.run("Indian")
print(answer)

Spice Haven


In [62]:
print(chain.memory.buffer)

Human: Italian
AI: La Bella Cucina
Human: Indian
AI: Spice Haven


In [64]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=llm)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [65]:
convo.run("Who won the first football worldcup?")

"The first football World Cup was won by Uruguay in 1930. The inaugural tournament was held in Uruguay, and the host nation triumphed by defeating Argentina 4-2 in the final match on July 30, 1930. This victory marked the beginning of what has become one of the most prestigious competitions in all of sports. Uruguay's win was celebrated with much fervor, especially since it coincided with the centenary of their first constitution."

In [68]:
print(convo.memory.buffer)

Human: Who won the first football worldcup?
AI: The first football World Cup was won by Uruguay in 1930. The inaugural tournament was held in Uruguay, and the host nation triumphed by defeating Argentina 4-2 in the final match on July 30, 1930. This victory marked the beginning of what has become one of the most prestigious competitions in all of sports. Uruguay's win was celebrated with much fervor, especially since it coincided with the centenary of their first constitution.


In [73]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

In [74]:
convo = ConversationChain(llm=llm, memory=memory)
convo.run("Who won the first football worldcup?")

"The first FIFA World Cup was held in 1930, and the winner was Uruguay. The tournament took place in Uruguay’s capital, Montevideo, with the final match held at the Estadio Centenario. Uruguay defeated Argentina 4-2 in the final to claim the championship. This event marked the beginning of what would become the world's most prestigious soccer competition."

In [75]:
convo.run("What is 5 + 5")

"5 + 5 equals 10. It's one of the basic addition facts that most people learn early on, and it’s quite handy in everyday situations! If you have any other questions or need further explanations, feel free to ask!"

In [76]:
convo.run("Who was the winning team country?")

'The winning team country of the first FIFA World Cup in 1930 was Uruguay. They triumphed over Argentina with a score of 4-2 in the final match, which took place at the Estadio Centenario in Montevideo, Uruguay. This victory made Uruguay the inaugural champions of the FIFA World Cup. If you have any other questions or need more details, feel free to ask!'